In [1]:
from gensim.models import KeyedVectors
from keras.layers.core import Dense, Dropout, SpatialDropout1D
from keras.layers.convolutional import Conv1D
from keras.layers.embeddings import Embedding
from keras.layers.pooling import GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import collections
import matplotlib.pyplot as plt
import nltk
import numpy as np
np.random.seed(42)

C:\ProgramData\Anaconda3\lib\site-packages\gensim-3.2.0-py3.6-win-amd64.egg\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
INPUT_FILE = "./data/umich-sentiment-train.txt"
WORD2VEC_MODEL = "./data/GoogleNews-vectors-negative300.bin.gz"
VOCAB_SIZE = 5000
EMBED_SIZE = 300
NUM_FILTERS = 256
NUM_WORDS = 3
BATCH_SIZE = 64
NUM_EPOCHS = 10

In [22]:
counter = collections.Counter() #  dictionary containing keys and counts
fin = open(INPUT_FILE, "r",encoding="utf8")
maxlen = 0
for line in fin:
    _, sent = line.strip().split("\t") # 1st argument neglected, only 2nd argument is send to sent(sentence)
    words = [x.lower() for x in nltk.word_tokenize(sent)]  # Splitting the sentence into words
    
    # ['oh', ',', 'and', 'brokeback', 'mountain', 'was', 'a', 'terrible', 'movie', '.']
    # len(words) -->10
    if len(words) > maxlen: # words --> contains array of words in a sentence that is splitted
        maxlen = len(words)
    for word in words:
        counter[word] += 1 # counter-> A dictionary store how many time a word appears in the whole test for "The:7"
        # "The" --> Occurs seven times
fin.close()


In [10]:
word2index = collections.defaultdict(int)
for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
    word2index[word[0]] = wid + 1
vocab_sz = len(word2index) + 1
index2word = {v:k for k, v in word2index.items()}

In [15]:
xs, ys = [], []
fin = open(INPUT_FILE, "rb",encoding="utf8")
for line in fin:
    label, sent = line.strip().split("\t")
    ys.append(int(label))
    words = [x.lower() for x in nltk.word_tokenize(sent)]
    wids = [word2index[word] for word in words]
    xs.append(wids)
fin.close()
X = pad_sequences(xs, maxlen=maxlen)
Y = np_utils.to_categorical(ys)

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3, 
                                                random_state=42)
print(Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape)

(4960, 42) (2126, 42) (4960, 2) (2126, 2)


In [17]:
# load word2vec model
word2vec = KeyedVectors.load_word2vec_format(WORD2VEC_MODEL, binary=True)
embedding_weights = np.zeros((vocab_sz, EMBED_SIZE))
for word, index in word2index.items():
    try:
        embedding_weights[index, :] = word2vec[word]
    except KeyError:
        pass

In [28]:
model = Sequential()
model.add(Embedding(vocab_sz, EMBED_SIZE, input_length=maxlen,
                    weights=[embedding_weights],
                    trainable=True))
# model.add(SpatialDropout1D(Dropout(0.2)))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=NUM_WORDS,
                 activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["accuracy"])
history = model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    validation_data=(Xtest, Ytest))          

Train on 4960 samples, validate on 2126 samples
Epoch 1/10
4960/4960 [==============================] - 13s 3ms/step - loss: 0.1192 - acc: 0.9542 - val_loss: 0.0210 - val_acc: 0.9934
Epoch 2/10
4960/4960 [==============================] - 14s 3ms/step - loss: 0.0095 - acc: 0.9976 - val_loss: 0.0152 - val_acc: 0.9958
Epoch 3/10
4960/4960 [==============================] - 14s 3ms/step - loss: 0.0039 - acc: 0.9994 - val_loss: 0.0136 - val_acc: 0.9953
Epoch 4/10
4960/4960 [==============================] - 14s 3ms/step - loss: 0.0020 - acc: 0.9998 - val_loss: 0.0127 - val_acc: 0.9958
Epoch 5/10
4960/4960 [==============================] - 15s 3ms/step - loss: 0.0027 - acc: 0.9996 - val_loss: 0.0137 - val_acc: 0.9972
Epoch 6/10
4960/4960 [==============================] - 15s 3ms/step - loss: 0.0014 - acc: 0.9998 - val_loss: 0.0159 - val_acc: 0.9972
Epoch 7/10
4960/4960 [==============================] - 15s 3ms/step - loss: 0.0020 - acc: 0.9996 - val_loss: 0.0142 - val_acc: 0.9967
Epoch 8

In [ ]:
# plot loss function
plt.subplot(211)
plt.title("accuracy")
plt.plot(history.history["acc"], color="r", label="train")
plt.plot(history.history["val_acc"], color="b", label="validation")
plt.legend(loc="best")

plt.subplot(212)
plt.title("loss")
plt.plot(history.history["loss"], color="r", label="train")
plt.plot(history.history["val_loss"], color="b", label="validation")
plt.legend(loc="best")

plt.tight_layout()
plt.show()

In [ ]:
# evaluate model
score = model.evaluate(Xtest, Ytest, verbose=1)
print("Test score: {:.3f}, accuracy: {:.3f}".format(score[0], score[1]))